In [1]:
import pandas as pd

In [2]:
def remove_duplicates(list_):
    collect = set()
    for item in list_:
        if item not in collect:
            collect.add(item)
    collect = list(collect)
    return collect

In [3]:
movie_reviews = pd.read_csv('IMDB Dataset.csv').filter(['review'])

reviews = list(movie_reviews['review'])
reviews = remove_duplicates(reviews)

In [4]:
len(reviews)

49582

In [6]:
movie_synopses = pd.read_csv(r'movies_metadata.csv', low_memory=False).filter(['overview'])

synopses = list(movie_synopses['overview'])
synopses = remove_duplicates(synopses)

In [7]:
reviews = list(filter(lambda x: not pd.isna(x), reviews))
synopses = list(filter(lambda x: not pd.isna(x), synopses))

In [8]:
print(len(reviews), len(synopses))

49582 44307


In [9]:
# down sample reviews
reviews = reviews[:len(synopses)]

In [10]:
len(reviews) == len(synopses), len(reviews), len(synopses)

(True, 44307, 44307)

In [11]:
data = reviews + synopses
len(data), len(data)/2, len(synopses), len(reviews)

(88614, 44307.0, 44307, 44307)

In [12]:
# reduce no. characters to 240 -- like Twitter, & speeds up training
data = [text[:240] for text in data]
len(data)

88614

In [17]:
# size = len(data)
labeled_reviews = [(data[i], 'Subjective') for i in range(len(reviews))]
labeled_synopses = [(data[i], 'Objective') for i in range(len(synopses))]

In [18]:
size = len(data)
labeled_data = []

for i in range(size):
    if i < size/2:
        labeled_data.append((data[i], 'Subjective'))
    else:
        labeled_data.append((data[i], 'Objective'))

In [19]:
labeled_data = remove_duplicates(labeled_data)
len(labeled_data)

88588

In [15]:
# len(labeled_reviews), len(labeled_synopses), len(labeled_reviews) + len(labeled_synopses)

In [14]:
# duplicates occur again, some text are indentical for the first 240 characters
# labeled_reviews = remove_duplicates(labeled_reviews)
# labeled_synopses = remove_duplicates(labeled_synopses)

In [20]:
len(labeled_reviews), len(labeled_synopses), len(labeled_reviews) + len(labeled_synopses)

(44307, 44307, 88614)

In [16]:
# labeled_synopses = labeled_synopses[:len(labeled_reviews)]
# len(labeled_reviews), len(labeled_synopses), len(labeled_reviews) + len(labeled_synopses)

(44283, 44283, 88566)

In [17]:
# from random import shuffle
# shuffle(labeled_reviews)
# shuffle(labeled_synopses)

In [18]:
# labeled_data = labeled_reviews + labeled_synopses

In [22]:
from random import shuffle
shuffle(labeled_data)

In [23]:
df = pd.DataFrame(labeled_data)

In [24]:
df.head()

,0,1
0,I found myself getting increasingly angry as t...,Subjective
1,A group of 7 youngsters-4 guys and 3 girls dec...,Objective
2,The ultimate X-Men ensemble fights a war for t...,Objective
3,"Irene, a thirty-something housewife with a suc...",Objective
4,A cop investigates the shooting of another pol...,Objective


In [25]:
df.tail()

,0,1
88583,Did this gem go direct to video? Fabulous art ...,Subjective
88584,"and there are not many in cinema history. ""Rou...",Subjective
88585,A 24 hour period in the lives of Fausto and Je...,Objective
88586,Ten people are invited for a weekend by a Mr U...,Objective
88587,A disabled man yearns for intimacy in a world ...,Objective


In [26]:
df = df.rename(columns={0:'Text', 1:'Labels'})

In [27]:
df.head()

,Text,Labels
0,I found myself getting increasingly angry as t...,Subjective
1,A group of 7 youngsters-4 guys and 3 girls dec...,Objective
2,The ultimate X-Men ensemble fights a war for t...,Objective
3,"Irene, a thirty-something housewife with a suc...",Objective
4,A cop investigates the shooting of another pol...,Objective


In [28]:
df['Text'].shape

(88588,)

In [29]:
import numpy as np
from sklearn.model_selection import train_test_split

In [30]:
# split data into train, validtion, and test sets

X_train, X_rem, y_train, y_rem = train_test_split(df['Text'], df['Labels'], train_size = 0.8, random_state = 24)

X_valid, X_test, y_valid, y_test = train_test_split(X_rem, y_rem, test_size = 0.5, random_state = 24)

In [ ]:
from transformers import AutoTokenizer

In [ ]:
# use roberta base sentiment model
rbs_model = f'cardiffnlp/twitter-roberta-base-sentiment'
tokenizer = AutoTokenizer.from_pretrained(rbs_model)

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(rbs_model)
model.save_pretrained(rbs_model)
tokenizer.save_pretrained(rbs_model)